In [3]:
DECLARE @payPeriod AS INT
SET @payPeriod = 202406

-- national capital region 
-- uses State and County codes in the NCR as defined in OPMs FAQ
DROP TABLE IF EXISTS #tempNCR
SELECT DISTINCT uupic
INTO #tempNCR
FROM pdw_dev_trusted.personal_organization_data
WHERE 1=1

    AND current_pay_period = @payPeriod

    AND (

        duty_station LIKE '11%001' OR
        duty_station LIKE '24%031' OR
        duty_station LIKE '24%033' OR
        duty_station LIKE '51%013' OR
        duty_station LIKE '51%059' OR
        duty_station LIKE '51%107' OR
        duty_station LIKE '51%153' OR
        duty_station LIKE '510040%' OR
        duty_station LIKE '510900%' OR
        duty_station LIKE '510930%' OR
        duty_station LIKE '511549%' OR
        duty_station LIKE '511550%'

    )

-- NCR teleworkers
DROP TABLE IF EXISTS #tempNCR_TW
SELECT DISTINCT uupic
INTO #tempNCR_TW
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND telework_flag_indicator = 'Y'
    AND remote_worker_indicator = 'N'
    AND uupic IN (SELECT uupic FROM #tempNCR)
    AND yrpp = @payPeriod

-- assigned to HQ
DROP TABLE IF EXISTS #tempHQ
SELECT DISTINCT uupic
INTO #tempHQ
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND center_text LIKE 'HQ'
    AND yrpp = @payPeriod

-- OIG
DROP TABLE IF EXISTS #tempOIG
SELECT DISTINCT uupic
INTO #tempOIG
FROM pdw_dev_trusted.personal_organization_data
WHERE 1=1
    AND center LIKE 'OIG'
    AND current_pay_period = @payPeriod

/* ############################################################## */

SELECT DISTINCT payperiod_end_date as PAY_PERIOD_END_DATE
FROM webtads_dev_trusted.daily_charges
WHERE yrpp = @payPeriod

-- verify headcount for civil servants onboard
-- D. Total # of Employees On Board at End of Pay Period, Nationally

SELECT COUNT(DISTINCT uupic) as val, 1 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND dutystatus IN ('A', 'L') 


UNION

-- total number of employees working remotely at the end of the pay period, nationally
-- E. Total # of Employee Working Remotely at End of Pay Period, Nationally

SELECT COUNT(DISTINCT uupic) as val, 2 as col
FROM webtads_dev_trusted.telework_requests
WHERE current_pay_period = @payPeriod
    AND duty_status IN ('Active', 'LWOP') 
    AND remote_agreement = 'Y'

UNION

-- # of employees on board in the National Capital Region
-- F. Total # of Employees On Board at end of Pay Period, in the National Capital Region (NCR)

SELECT COUNT(DISTINCT uupic) as val, 3 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND dutystatus IN ('A', 'L') 
    and uupic IN (SELECT uupic FROM #tempNCR)

UNION

-- verify # of employees teleworked during the pay period, nationally
-- G. Total # of Employees who Teleworked During the Pay Period, Nationally

SELECT COUNT(DISTINCT uupic) as val,  4 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND dutystatus IN ('A', 'L') 
    AND telework_flag_indicator = 'Y' 
    AND remote_worker_indicator = 'N'

UNION

-- # of employees who teleworked during the pay period, National Capital Region
-- H. Total # of Employees who Teleworked During Pay Period, in the NCR

SELECT COUNT(*) as val,  5 as col
FROM #tempNCR_TW

UNION

-- # of employees assigned to Agency HQ and HQ-Equivalent organizations (taken to mean HQ center assignment in PDW)
-- I. Total # of Employees  Assigned to Agency HQ and HQ-Equivalent Organizations

SELECT COUNT(DISTINCT uupic) as val, 6 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND center_text = 'HQ'
    AND yrpp = @payPeriod

UNION

-- validate regular payroll hours worked nationally
-- J. Total # of Regular Payroll Hours - All Employees, Nationally

SELECT SUM(a.hours_charged) as val, 7 as col
FROM webtads_dev_trusted.daily_charges as a
LEFT JOIN webtads_dev_trusted.leave_hour_types as b
ON a.hour_type = b.[HourType Name]
WHERE 1=1
    AND a.yrpp = @payPeriod
    AND a.hour_type LIKE 'REG%' 

UNION

-- validate regular payroll hours coded as telework worked nationally
-- K. Total # of Regular Hours Coded as Telework - All Employees, Nationally

SELECT SUM(hours_charged) as val, 8 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'
    AND telework_flag_indicator = 'Y' 
    AND remote_worker_indicator = 'N'

UNION

-- L. % of Hours Performed In-Person - All Employees, Nationally 
SELECT NULL as val, 9 as col
FROM webtads_dev_trusted.daily_charges

UNION

-- total regular hours in NCR
-- M. Total # of Regular Payroll Hours - Employees with Duty Stations in the NCR

SELECT SUM(hours_charged) as val,  10 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'  
    AND uupic IN (SELECT uupic FROM #tempNCR)

UNION

-- telework hours for NCR
-- N. Total # of Regular Hours Coded as Telework - Employees with Duty Stations in the NCR

SELECT SUM(hours_charged) as val, 11 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'
    AND telework_flag_indicator = 'Y' 
    AND remote_worker_indicator = 'N'
    AND uupic IN (SELECT uupic FROM #tempNCR)

UNION

-- O. % of Hours Performed In-Person - Employees with Duty Stations in the NCR
SELECT NULL as val, 12 as col
FROM webtads_dev_trusted.daily_charges

UNION

-- total regular hours in NCR, teleworkers only
-- P. Total # of Regular Payroll Hours - Employees with Duty Stations in the NCR who Teleworked in the Pay Period

SELECT SUM(hours_charged) as val, 13 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'  
    AND uupic IN (SELECT uupic FROM #tempNCR_TW)

UNION

-- telework hours
-- Q. Total # of Regular Hours Coded as Telework - Employees with Duty Stations in the NCR who Teleworked in the Pay Period

SELECT SUM(hours_charged) as val, 14 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'  
    AND telework_flag_indicator = 'Y' 
    AND remote_worker_indicator = 'N'
    AND uupic IN (SELECT uupic FROM #tempNCR_TW)

UNION

--. R. % of Hours Performed In-Person - Employees with Duty Stations in the NCR who Teleworked in the Pay Period
SELECT NULL as val, 15 as col
FROM webtads_dev_trusted.daily_charges

UNION

-- total regular hours for HQ orgs
-- S. Total # of Regular Payroll Hours - Employees Assigned to Agency HQ and HQ-Equivalent Organizations

SELECT SUM(hours_charged) as val,  16 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'  
    AND center_text = 'HQ'

UNION

-- telework hours for  HQ
-- T. Total # of Regular Hours Coded as Telework - Employees Assigned to Agency HQ and HQ-Equivalent Organizations

SELECT SUM(hours_charged) as val, 17 as col
FROM webtads_dev_trusted.daily_charges
WHERE 1=1
    AND yrpp = @payPeriod
    AND hour_type LIKE 'REG%'
    AND telework_flag_indicator = 'Y'
    AND remote_worker_indicator = 'N'
    AND center_text = 'HQ'

ORDER BY col ASC

(3878 rows affected)

(2844 rows affected)

(1814 rows affected)

(178 rows affected)

(1 row affected)

(17 rows affected)

Total execution time: 00:00:12.062

PAY_PERIOD_END_DATE
2024-03-09


val,col
18382,1
2210,2
3873,3
12296,4
2844,5
1814,6
1315748.75,7
453128.75,8
NULL,9
278249.5,10
